In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import holidays
import streamlit as st
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import joblib

In [3]:
inferencia_data = r"D:\forecasting_retailer\data\raw\inference\ventas_2025_inferencia.csv"
inferencia_df = pd.read_csv(inferencia_data)

In [4]:
# ...código existente...
# 1. Conversión de fecha
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Variables temporales
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.weekday
inferencia_df['nombre_dia'] = inferencia_df['fecha'].dt.day_name(locale='es_ES')

# 3. Es fin de semana
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5, 6])

# 4. Es festivo (España)
festivos_es = []
for year in inferencia_df['año'].unique():
    festivos_es += [d for d in holidays.country_holidays('ES', years=[year]).keys()]
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es)

# 5. Black Friday y Cyber Monday
def es_black_friday(fecha):
    if fecha.month == 11:
        ultimo_viernes = max([d for d in pd.date_range(start=fecha.replace(day=1), end=fecha.replace(day=30)) if d.weekday() == 4])
        return fecha == ultimo_viernes
    return False

def es_cyber_monday(fecha):
    if fecha.month == 11:
        ultimo_viernes = max([d for d in pd.date_range(start=fecha.replace(day=1), end=fecha.replace(day=30)) if d.weekday() == 4])
        cyber_monday = ultimo_viernes + pd.Timedelta(days=3)
        return fecha == cyber_monday
    return False

inferencia_df['es_black_friday'] = inferencia_df['fecha'].apply(es_black_friday)
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# 6. Semana del año, trimestre, día del año
inferencia_df['semana_anio'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['dia_anio'] = inferencia_df['fecha'].dt.dayofyear

# 7. Es inicio y fin de mes
inferencia_df['es_inicio_mes'] = inferencia_df['dia_mes'] == 1
inferencia_df['es_fin_mes'] = inferencia_df['fecha'].dt.is_month_end

# 8. Variables de competencia y ratio_precio
# Si existen las columnas de competencia, calcular precio_competencia y ratio_precio
competencia_cols = ['Amazon', 'Decathlon', 'Deporvillage']
if all(col in inferencia_df.columns for col in competencia_cols):
    inferencia_df['precio_competencia'] = inferencia_df[competencia_cols].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
else:
    inferencia_df['precio_competencia'] = np.nan
    inferencia_df['ratio_precio'] = np.nan

# 9. Descuento
inferencia_df['descuento'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 10. Lags y media móvil (por producto)
for lag in range(1, 8):
    inferencia_df[f'lag{lag}'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(lag)
inferencia_df['media_movil_7d'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].rolling(window=7, min_periods=1).mean().reset_index(level=0, drop=True)

# 11. OneHotEncoding de nombre, categoria y subcategoria (con sufijo _h)
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']
inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre_h', 'categoria_h', 'subcategoria_h'], drop_first=True)

# 12. Eliminar registros de octubre, dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].reset_index(drop=True)

C:\Users\pablo\AppData\Local\Temp\ipykernel_8576\4148959666.py:19: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es)


In [5]:
# 13. Guardar el dataframe transformado
inferencia_df.to_csv(r"D:\forecasting_retailer\data\processed\inferencia_df_transformado.csv", index=False)